In [1]:
# 01-30-2025

# HANDS-ON LAB: Acquiring and Processing Information on the World's Largest Banks

# Project Scenario:
#   You have been hired as a data engineer by research organization. Your boss has asked you 
# to create a code that can be used to compile the list of the top 10 largest banks in the world 
# ranked by market capitalization in billion USD. Further, the data needs to be transformed and stored 
# in GBP, EUR, and INR as well, in accordance with the exchange rate information that has been made 
# available to you as a CSV file. The processed information table is to be saved locally in a CSV
# format and as a database table.

#   Your job is to create an automated system to generate this information so that the same can be 
# executed in every financial quarter to prepare the report.

# Particulars of the code to be made have been shared below.
# Code name : banks_project.py
# Data URL : https://en.wikipedia.org/wiki/List_of_largest_banks
# Exchange rate CSV : https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv
# Table attributes (upon extraction only) : Name, MC_USD_Billion
# Table Attributes (final) : Name, MC_USD_Billion, MC_GBP_Billion, MC_EUR_Billion, MC_INR_Billion
# Output CSV Path : ./Largest_banks_data.csv
# Database name : Banks.db
# Table name : Largest_banks
# Log file : code_log.txt

## Task 1:
#           Write a function log_progress() to log the progress of the code at different stages in a file
#       code_log.txt. Use the list of log points provided to create log entries as every stage of the code.

## Task 2:
#           Extract the tabular information from the given URL under the heading 'By Market Capitalization
#       and save it to a dataframe.
#           a. inspect the webage and indentify the position and pattern of the tabular information in the HTML code
#           b. Write the code for a function extract()
#           c. Execute a function call to extract() to verify the output

## Task 3:
#           Transform the dataframe by adding columns for Market Capitalization in GBP, EUR, and INR, rounded to 2
#       decimal places, based on the exchange rate information shared in CSV file.
#           a. Write the code for a function transform() to perform the said task.
#           b. Execute a function call to transform() and verify the output.

## Task 4:
#           Load the transformed dataframe to an output CSV file. Write a function load_to_csv(), execute a function
#       call and verify the output.

## Task 5:
#           Load the transformed dataframe to an SQL database server as a table. Write a function load_to_db(),
#       execute a function call and verify the output.

## Task 6:
#           Run queries on the database table. Write a function load_to_db(), execute a given set of queries and
#       verify the output.

## Task 7:
#           Verify that the log entries have been completed at all stages by checking the contents of the file
#       code_log.txt.



In [1]:
import requests
import pandas as pd 
import numpy as np 
import sqlite3 
from datetime import datetime 
from bs4 import BeautifulSoup

In [ ]:
log_file = 'code_log.txt'
exchange_rate = 'exchange_rate.csv'
data_url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'

# Task 1:
def log_progress(log_msg):
    timestamp_fmt = '%Y-%m-%d-%H:%M:%S' # timestamp format year-month-date-hour-min-sec
    crnt_time = datetime.now() # current time
    timestamp = crnt_time.strftime(timestamp_fmt)
    with open(log_file, 'a') as log_f:
        log_f.write(timestamp + ' : ' + log_msg + '\n')

def extract(url):
    url_get = requests.get(url)
    soup = BeautifulSoup(url_get.text, 'html.parser')
    table = soup.find_all('table')[0]
    table_headers = table.find_all('th')
    bank_table_titles = []
    for titles in table_headers:
        bank_table_titles.append(titles.text.strip())
    df = pd.DataFrame(columns=bank_table_titles)
    column_data = table.find_all('tr')
    for row in column_data[1:]:
        bank_row_data = row.find_all('td')
        per_row_bank_data = [data.text.strip() for data in bank_row_data]
        df_length = len(df)
        df.loc[df_length] = per_row_bank_data
    return df

def transform(df, csv_path):
    df_2 = pd.read_csv(exchange_rate)
    dict_df2_currency = df_2.set_index('Currency').to_dict()['Rate']
    df.rename(columns={'Market cap(US$ billion)' : 'MC_USD_Billion' }, inplace=True)
    df['MC_GBP_Billion'] = [np.round(x*dict_df2_currency['GBP'],2) for x in df['MC_USD_Billion'].astype(float)]
    df['MC_EUR_Billion'] = [np.round(x*dict_df2_currency['EUR'],2) for x in df['MC_USD_Billion'].astype(float)]
    df['MC_INR_Billion'] = [np.round(x*dict_df2_currency['INR'],2) for x in df['MC_USD_Billion'].astype(float)]
    return df

def load_to_csv(df, saveto_csv):
    df.to_csv(saveto_csv)

def load_to_db(df, sql_connection, table_name):
    sql_ = df.to_sql(table_name, sql_connection, if_exists = 'replace', index=False)
    return sql_

def run_queries(query_statement, sql_connection):
    queries = pd.read_sql(query_statement, sql_connection)
    return queries


log_progress('Preliminaries complete. Initiating ETL process')

x = extract(data_url)

log_progress('Data extraction complete. Initiating Transformation process')

y = transform(x, exchange_rate)

log_progress('Data transformation complete. Initiating Loading process')

z = load_to_csv(y, 'bank_mc.csv')

log_progress('Data saved to CSV file')

log_progress('SQL connection initiated')

conn = sqlite3.connect('Banks.db')
table_name = 'Largest_Banks'
load_to_db(y, conn, table_name)

log_progress('Data loaded to Database as a table, executing queries ....')

statement = 'SELECT * from Largest_banks LIMIT 5'
sql_query = run_queries(statement, conn)
print(sql_query)
statement = 'SELECT AVG(MC_GBP_Billion) from Largest_banks LIMIT 5'
sql_query = run_queries(statement, conn)
print(sql_query)
statement = 'SELECT [Bank name] from Largest_banks LIMIT 5'
sql_query = run_queries(statement, conn)
print(sql_query)

log_progress('Process complete')

log_progress('Server connection closed')

  Rank                                Bank name MC_USD_Billion  \
0    1                           JPMorgan Chase         432.92   
1    2                          Bank of America         231.52   
2    3  Industrial and Commercial Bank of China         194.56   
3    4               Agricultural Bank of China         160.68   
4    5                                HDFC Bank         157.91   

   MC_GBP_Billion  MC_EUR_Billion  MC_INR_Billion  
0          346.34          402.62        35910.71  
1          185.22          215.31        19204.58  
2          155.65          180.94        16138.75  
3          128.54          149.43        13328.41  
4          126.33          146.86        13098.63  
   AVG(MC_GBP_Billion)
0              151.987
                                 Bank name
0                           JPMorgan Chase
1                          Bank of America
2  Industrial and Commercial Bank of China
3               Agricultural Bank of China
4                             